In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
print(df)

NameError: name 'df' is not defined

In [3]:
df = pd.read_csv('data/2017_00_matchup_recap.txt', header=0)

scores = df.loc[df['SEASON']==2017][['TEAMNAME','SCORINGPERIOD','SCORE','OUTCOME']]

scores['ACTUAL'] = scores.apply(lambda row: row.OUTCOME == 'W', axis=1)
scores['EXPECTED'] = scores['SCORE'].to_frame().apply(st.zscore, ddof=1).apply(st.norm.cdf)
scores['EXPECTED_GT50'] = scores.apply(lambda row: row.EXPECTED > .5 , axis=1)
scores['EXPECTED_AVG'] = (scores['EXPECTED']+scores['EXPECTED_GT50'])/2

exp_wins = scores.groupby('TEAMNAME').sum()[['ACTUAL','EXPECTED','EXPECTED_GT50','EXPECTED_AVG']]
exp_wins['LUCK']=exp_wins['ACTUAL']-exp_wins['EXPECTED']
exp_wins['LUCK_GT50']=exp_wins['ACTUAL']-exp_wins['EXPECTED_GT50']
exp_wins['LUCK_AVG']=(exp_wins['LUCK']+exp_wins['LUCK_GT50'])/2



In [22]:
exp_wins[['ACTUAL','EXPECTED_AVG','LUCK_AVG']].sort_values('LUCK_AVG')

,ACTUAL,EXPECTED_AVG,LUCK_AVG
TEAMNAME,,,
Jeremy,7.0,9.293621,-2.293621
Tony,3.0,5.199524,-2.199524
Thomas,4.0,5.439241,-1.439241
Schwartz,6.0,7.243049,-1.243049
Scott,8.0,9.103900,-1.103900
Paul,8.0,8.648714,-0.648714
Tim,7.0,6.788894,0.211106
JMT,9.0,8.346192,0.653808
JJ,7.0,6.086076,0.913924


In [28]:
scores[scores['SCORINGPERIOD']==scores['SCORINGPERIOD'].max()][['TEAMNAME','SCORE','OUTCOME','EXPECTED_AVG']].sort_values('EXPECTED_AVG', ascending=False)

,TEAMNAME,SCORE,OUTCOME,EXPECTED_AVG
164,Brent,160.6,W,0.996326
162,Kyle,143.0,W,0.972398
156,Jeremy,128.0,W,0.904949
161,JMT,116.6,W,0.814878
167,Tim,111.6,W,0.768245
165,Paul,106.9,L,0.223422
157,JJ,101.7,L,0.175460
158,Scott,96.3,W,0.130280
160,Schwartz,94.7,L,0.118144
166,Goss,92.8,L,0.104579


In [29]:
h2h = df.groupby(['TEAMNAME','OPPONENTNAME', 'OUTCOME'])['OUTCOME'].count().unstack().fillna(0)

<bound method NDFrame.keys of OUTCOME                  L    T    W
TEAMNAME OPPONENTNAME               
Brent    Goss          1.0  0.0  1.0
         JJ            0.0  0.0  1.0
         JMT           1.0  0.0  0.0
         Jeremy        0.0  0.0  1.0
         Kyle          1.0  0.0  0.0
         Paul          0.0  0.0  2.0
         Schwartz      1.0  0.0  0.0
         Scott         0.0  0.0  1.0
         Thomas        0.0  0.0  1.0
         Tim           1.0  0.0  1.0
         Tony          0.0  0.0  1.0
Goss     Brent         1.0  0.0  1.0
         JJ            0.0  0.0  1.0
         JMT           0.0  0.0  1.0
         Jeremy        1.0  0.0  0.0
         Kyle          1.0  0.0  0.0
         Paul          1.0  0.0  1.0
         Schwartz      1.0  0.0  0.0
         Scott         1.0  0.0  0.0
         Thomas        1.0  0.0  0.0
         Tim           1.0  0.0  1.0
         Tony          0.0  0.0  1.0
JJ       Brent         1.0  0.0  0.0
         Goss          1.0  0.0  0.0
        